In [1]:
from trialblazer.Dataset_preprocess.name_extraction_Smilesmatching import ExtractingDrugsNamesAndSmilesMatching
import pandas as pd

[10:05:06] Initializing Normalizer


## Training data

### Benign data_Approved drug

In [ ]:
DrugBank_approved_dropnan = pd.read_csv('../datasets/approved/DrugBank/check_nan.csv', sep='\t')

In [ ]:
Approved_drugs_dropnan_SmilesMatched, GoTo_NextRound = ExtractingDrugsNamesAndSmilesMatching(DrugBank_approved_dropnan,'Name')

### Benign data_ClinicalTrial

In [51]:
def phase_select(phase):
        if "Phase 3" in phase and "Phase 4" in phase:
                return "Phase 4"
        else:
                return phase

In [ ]:
Benign_annotated = pd.read_csv("../datasets/data_aact/Pipe-Delimited/check_other_phase_1/bad,good,multi_drugs/good_part_process/df1_goodpart_annotated_20231205.csv")

In [19]:
Benign_annotated = Benign_annotated.drop(columns=['replace_Name','DNER_with_spacy'])

In [ ]:
ClinicalTrial_benign_SmilesMatched, GoTo_NextRound = ExtractingDrugsNamesAndSmilesMatching(Benign_annotated,'Name-Huanni')

phase selecting: only use phase 3 and phase 4 drugs

In [ ]:
phase_NeedToExcluded = ['Phase 2/Phase 3', 'SPD489']
Benign_annotated_phase_filtered = ClinicalTrial_benign_SmilesMatched.loc[ClinicalTrial_benign_SmilesMatched['phase'].str.contains('|'.join(phase_NeedToExcluded)) == False]
Benign_annotated_phase_filtered['phase']=  Benign_annotated_phase_filtered['phase'].apply(phase_select)

### Toxic data_clinicalTrial

In [ ]:
toxic_drugs = pd.read_csv("../datasets/data_aact/Pipe-Delimited/check_other_phase_1/bad,good,multi_drugs/toxic_part_process/toxicity_onedrug.csv", sep='|')
toxic_drugs.drop(columns=['replace_Name', 'DNER_with_spacy', 'name_to_use', 'synonyms',	'drugbank_id'], inplace = True)

In [ ]:
ClinicalTrial_benign_SmilesMatched, GoTo_NextRound = ExtractingDrugsNamesAndSmilesMatching(toxic_drugs,'Name_check')

after manually curated

In [ ]:
clinical_trial_toxic = pd.read_csv("../datasets/toxic_pool/clinical_trial_toxic.csv", sep='|')

### Toxic data_other than clinical trial data

Load prepared withdrawn dataset

In [ ]:
withdrawn = pd.read_csv("../datasets/toxic_pool_recheck/withdrawn.csv")
withdrawn.rename(columns={'SMILES':'SMILES_ORI'}, inplace=True)

In [ ]:
Withdrawn_toxic_SmilesMatched, GoTo_NextRound = ExtractingDrugsNamesAndSmilesMatching(withdrawn,'Name')

After manual curation

In [ ]:
withdrawn_SmilesMatched = pd.read_csv('../toxic_withdrawn_organized/withdrawn_SmilesMatched.csv', sep='|')
withdrawn_manual_add = pd.read_csv('../toxic_withdrawn_organized/GoTo_NextRound_Manual_check.csv', sep='|')

In [11]:
manually_curated_SMILES = withdrawn_manual_add.loc[withdrawn_manual_add['SMILES'] != '-']
Fail_to_curate_SMILES = withdrawn_manual_add.loc[withdrawn_manual_add['SMILES'] == '-']

In [12]:
print("length of manually curated SMILES:", len(manually_curated_SMILES))
print("length of compounds be removed:", len(Fail_to_curate_SMILES))

length of manually curated SMILES: 19
length of compounds be removed: 48


In [13]:
withdrawn_smiles = pd.concat([withdrawn_SmilesMatched, manually_curated_SMILES], ignore_index=True)

## Test set data

In [ ]:
EMA_approved = pd.read_csv("../external_test_set/EMA_approved.csv", sep='|')
KEGG_Japan_approved = pd.read_csv("../external_test_set/KEGG_Japan_approved.csv", sep='|')

In [ ]:
EMA_SmilesMatched, EMA_GoTo_NextRound = ExtractingDrugsNamesAndSmilesMatching(EMA_approved, 'Active substance')

In [ ]:
print("then number of the matched SMIELS from EMA:", len(EMA_SmilesMatched))
print("then number of the SMILES can not automatically match from EMA:", len(EMA_GoTo_NextRound))

In [ ]:
KEGG_Japan_SmilesMatched, KEGG_Japan_GoTo_NextRound = ExtractingDrugsNamesAndSmilesMatching(KEGG_Japan_approved, 'Active ingredient')

In [74]:
print("then number of the matched SMIELS from KEGG_Japan:", len(KEGG_Japan_SmilesMatched))
print("then number of the SMILES can not automatically match from KEGG_Japan:", len(KEGG_Japan_GoTo_NextRound))

then number of the matched SMIELS from KEGG_Japan: 1493
then number of the SMILES can not automatically match from KEGG_Japan: 651


Process the manually added SMILES notation

In [ ]:
EMA_SmilesMatched = pd.read_csv("../external_test_set/EMA_SmilesMatched.csv", sep='|')
KEGG_Japan_SmilesMatched= pd.read_csv("../external_test_set/KEGG_Japan_SmilesMatched.csv", sep='|')
EMA_SmilesMatched['resource'] = 'EMA'
KEGG_Japan_SmilesMatched['resource'] = 'KEGG'

In [22]:
EMA_SmilesMatched_sim = EMA_SmilesMatched[['Medicine name', 'SMILES','resource']]
KEGG_SmilesMatched_sim = KEGG_Japan_SmilesMatched[['Active ingredient', 'SMILES','resource']]

In [ ]:
unmatched_KEGG_sarah = pd.read_csv('../drugname_unmatched_KEGGandEMA/KEGG_Japan_unmatched_sarah.csv_with_smiles.txt', sep='\t', names=['Active ingredient','CID','SMILES'])
unmatched_EMA_sarah = pd.read_csv('../drugname_unmatched_KEGGandEMA/EMA_Japan_unmatched_sarah.csv_with_smiles.txt', sep='\t', names=['Medicine name','CID','SMILES'])
unmatched_KEGG_sarah_SMILES = unmatched_KEGG_sarah[unmatched_KEGG_sarah['SMILES'] != 'Not Found'][['Active ingredient','SMILES']]
unmatched_EMA_sarah_SMILES = unmatched_EMA_sarah[unmatched_EMA_sarah['SMILES'] != 'Not Found'][['Medicine name','SMILES']]

In [44]:
unmatched_KEGG_sarah_SMILES['resource'] = 'KEGG'
unmatched_EMA_sarah_SMILES['resource'] = 'EMA'

In [ ]:
EMA_concat = pd.concat([EMA_SmilesMatched_sim,unmatched_EMA_sarah_SMILES], ignore_index=True)
EMA_concat.rename(columns={'Medicine name':'Name'},inplace=True)

In [ ]:
KEGG_concat = pd.concat([KEGG_SmilesMatched_sim, unmatched_KEGG_sarah_SMILES], ignore_index=True)
KEGG_concat.rename(columns={'Active ingredient':'Name'}, inplace=True)

In [ ]:
Testset_Approved = pd.concat([EMA_concat,KEGG_concat], ignore_index=True)